## Leitura de arquivos

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [12]:
import findspark
findspark.init()

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('Inicializando com Spark').getOrCreate()

In [19]:
spark